# Langchain for Model, Prompt, Reusable Template and Output Parser

In [ ]:
# Install Packages
!pip install -U langchain langchain-community langchain-google-genai

In [ ]:
# Check Langchain Version
import langchain
print(langchain.__version__)

In [ ]:
import os
from google import genai

# 1. Setup your key
os.environ["GEMINI_API_KEY"] = "XXX"
# 2. Import the model
from langchain_google_genai import ChatGoogleGenerativeAI
# 3. Initialize the model 
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
# 4. Invoke the model
result = llm.invoke("What is 1+1")
print(result.content)


## Basic template

In [ ]:
customer_email = """
I am fuming over the spoiled machine"""

style = """Pirate English"""

prompt = f"""Translate the text \
    that is delimited by triple backticks
    into a style thst is {style}.
    text: ```{customer_email}```
    """

print(prompt)

In [ ]:
result = llm.invoke(prompt)
print(result.content)

## Reuse template with Langchain

In [ ]:
template_string = """Translate the text \
    that is delimited by triple backticks
    into a style thst is {style}.
    text: ```{text}```
    """

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [ ]:
customer_style = """American English in a calm \
                    and respectful tone"""

customer_email = """ \
    I am fuming over the spoiled machine. \
    Why did it breakdown in less than 3 months? \
    Where was this manufactured in? \
    What lousy parts were used?"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email
                    )


In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages)

In [ ]:
result = llm.invoke(customer_messages)
print(result.content)

## Defining Output Parser

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [2]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
import os
from google import genai

# 1. Setup your key (Using the one you provided)
os.environ["GEMINI_API_KEY"] = "XXX"

# 2. Import the model
from langchain_google_genai import ChatGoogleGenerativeAI

# 3. Initialize the model (e.g., gemini-1.5-pro or gemini-pro)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# 4. Invoke the model
result = llm.invoke("What is 1+1")
print(result.content)

from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)

messages = prompt_template.format_messages(
                    text=customer_review
                    )

result = llm.invoke(messages, temperature=0.0)
print (result.content)


1 + 1 = 2
```json
{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
```


In [4]:
from pydantic import BaseModel, Field
from typing import List
from langchain_google_genai import ChatGoogleGenerativeAI

# 1. Standard Pydantic definition
class ReviewAnalysis(BaseModel):
    gift: bool = Field(description="Was it a gift? True/False.")
    delivery_days: int = Field(description="Days to arrive, -1 if unknown.")
    price_value: List[str] = Field(description="Sentences about price.")

# 2. Modern 1.2.6 Implementation
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# This binds the schema to the model natively
structured_llm = llm.with_structured_output(ReviewAnalysis)

# 3. Execution (The output is already a Python object!)
result = structured_llm.invoke(messages)

print (result)
print (result.gift)
print (result.delivery_days)
print (result.price_value)

gift=True delivery_days=2 price_value=["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
True
2
["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]


In [6]:
import json

# Convert the Pydantic object to a standard Python dictionary and to JSON
formatted_result = result.model_dump()

print(json.dumps(formatted_result, indent=2))

{
  "gift": true,
  "delivery_days": 2,
  "price_value": [
    "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
  ]
}
